In [1]:
import sys
import os
import pandas as pd
import re
import requests
import itertools
from copy import deepcopy
from collections import namedtuple, defaultdict

from bs4 import BeautifulSoup as BS
from tenacity import retry, stop_after_attempt
import simplejson as json

In [2]:
session = requests.Session()
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6",
    "Connection": "keep-alive",
    "Referer": "https://www.google.com.tw/",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) "\
                  "Chrome/69.0.3497.92 Safari/537.36"
}

In [3]:
@retry(stop=stop_after_attempt(3))
def get_bsObj(url, headers=headers):
    req = session.get(url, headers=headers)
    bsObj = BS(req.text, "html.parser")
    return bsObj

In [10]:
year_issuekey_filename = os.path.join('temp', 'YearIssueKey.csv')

if os.path.exists(year_issuekey_filename):
    df_issuekeys = pd.read_csv(year_issuekey_filename)
else:

    all_issues = get_bsObj("https://www.cambridge.org/core/journals/behavioral-and-brain-sciences/all-issues")
    year_to_issues = defaultdict(list)
    for bsObj in all_issues.find_all('span', text=re.compile("[0-9]{4} \- Volume ")):
        year = int(bsObj.text[:4])
        block = bsObj.parent.parent.parent.find('ul', class_='accordion level fourth')
        for single_issue in block.find_all('a', class_='row'):
            year_to_issues[year].append(single_issue['href'].split('/')[-1])

    df_years = []
    df_keys  = []
    for year, issue_keys in year_to_issues.items():
        for issue_key in issue_keys:
            df_years.append(year)
            df_keys.append(issue_key)

    df_issuekeys = pd.DataFrame(data={'year': df_years, 'issue_key': df_keys})
    df_issuekeys.to_csv(year_issuekey_filename, index=False)

In [11]:
def issue_page_to_url(issue, page_num):
    url_format = 'https://www.cambridge.org/core/journals/behavioral-and-brain-sciences/' \
                 'issue/{}?pageNum={}'
    return url_format.format(issue, page_num)

def get_website_url(suffix):
    prefix = 'https://www.cambridge.org{}'
    return prefix.format(suffix)

In [12]:
ArticleInfo = namedtuple('ArticleInfo', ['authors', 'title', 'url', 'article_type', 'references'])
AuthorInfo = namedtuple('Author', ['author_name', 'url'])
DetailedName = namedtuple('DetailedName', ['surname', 'given_names'])
ReferenceInfo = namedtuple('ReferenceInfo', ['authors', 'year', 'title', 'source'])

In [13]:
def get_article_infos_from_issue(issue):
    
    article_infos = []
    for page_num in itertools.count(start=1):

        page = get_bsObj(issue_page_to_url(issue, page_num))

        heading_banner = page.find('h4', class_='journal-article-listing-type heading_12 margin-bottom')
        if not heading_banner:
            break
        assert heading_banner['class'] == ['journal-article-listing-type', 'heading_12', 'margin-bottom']

        now_type = heading_banner.text
        print('\n', now_type, end='\n\n')

        for idx, item in enumerate(heading_banner.find_next_siblings()):
            link = item.find('a', class_='part-link')
            authors = item.find('li', class_='author')
            if link and authors:
                author_infos = []
                for author_objs in authors.find_all('a'):
                    author_name = author_objs.text
                    author_url = author_objs['href']
                    author_infos.append(AuthorInfo(author_name=author_name, url=author_url))

                title = link.text.strip()
                url = get_website_url(link['href'])
#                 references = get_references_from_url(url)
                info = ArticleInfo(authors=author_infos,
                                   title=title,
                                   url=url,
                                   article_type=now_type,
                                   references=references)
                article_infos.append(info)

                print(title)

            else:
                if item['class'] != ['journal-article-listing-type', 'heading_12', 'margin-bottom']:
                    continue
                now_type = item.text
                print('\n', now_type, end='\n\n')
    return article_infos

In [24]:
def get_references_from_url(url):
    
    req = session.get(url, headers=headers)
    art_bsObj = BS(req.text, "html.parser")
    # art_bsObj

    reference_infos = []
    cc = art_bsObj.find('div', id='contentContainer')
    if cc is None:
        return []
    
    ref_list = cc.find('div', class_='ref-list')
    if not ref_list:
        return []
    
    refs = ref_list.find_all('div', class_='ref')
    for ref in refs:

        authors = []
        for name in ref.find_all('span', class_='name'):
            surname = name.find('span', class_='surname')
            surname = surname.text if surname else None

            given_names = name.find('span', class_='given-names')
            given_names = given_names.text if given_names else None

            detailed_name = DetailedName(surname=surname, given_names=given_names)
            authors.append(detailed_name)

        year = ref.find('span', class_='year')
        year = year.text if year else None

        title = ref.find('span', class_='article-title')
        title = title.text if title else None

        source = ref.find('span', class_='source')
        source = source.text if source else None

        reference_info = ReferenceInfo(authors=authors,
                                       year=year,
                                       title=title,
                                       source=source)
        reference_infos.append(reference_info)
    return reference_infos

In [ ]:
# def transform_list_to_trees(articles, head_types=['Target Article'], children_types=['Open Peer Commentary']):
#     head_to_children = dict()
    
#     head_article = None
    
#     article_types = set()
#     for article in articles:
        
#         article_types.add(article['article_type'])

#         if article['article_type'] in head_types:

#             _article = deepcopy(article)
#             title = _article.pop('title', None)
#             head_article = title
            

#         elif article['article_type'] in children_types:
            
#             if head_article is None:
#                 break

#             if title not in head_to_children:
#                 head_to_children[title] = _article
#                 head_to_children[title]['commentaries'] = []
                

#             head_to_children[head_article]['commentaries'].append(article)
        
#         else:
#             head_article = None

#     if not all(children_lists['commentaries'] for head, children_lists in head_to_children.items()):
#         print([head for head, children_lists in head_to_children.items() if children_lists['commentaries']])
#         print([head for head, children_lists in head_to_children.items() if not children_lists['commentaries']])
#         assert False
    
#     return head_to_children

In [91]:
for idx, (year, key) in df_issuekeys.iterrows():
    
    if idx < 125:
        continue
    
    folder_name = f'temp/{key}'
    os.makedirs(folder_name, exist_ok=True)
    
    articles = get_article_infos_from_issue(key)
    
    df_article_infos = []
    df_article_authors = []
    df_reference_infos = []
    
    # need to handle 'authors', 'title', 'url', 'article_type', 'references' of article_infos
    for a in articles:
        df_article_infos.append([year, key, a.title, a.url, a.article_type])

        # need to handle 'author_name', 'url' of authors
        for aut in a.authors:
            df_article_authors.append([a.url, aut.author_name, aut.url])

        # need to handle 'authors', 'year', 'title', 'source' of references
        for r in a.references:
            for aut in r.authors:
                df_reference_infos.append([a.url, r.title, r.year, r.source, aut.surname, aut.given_names])

    df_article_infos = pd.DataFrame(df_article_infos, columns=['year', 'issue_key', 'article_title', 'article_url', 'article_type'])
    df_article_authors = pd.DataFrame(df_article_authors, columns=['article_url', 'author_name', 'author_url'])
    df_reference_infos = pd.DataFrame(df_reference_infos, 
                                      columns=['article_url', 'reference_title', 'reference_year', 'reference_source',
                                               'reference_author_surname', 'reference_author_given_name'])
    
    
    df_article_infos.to_csv(os.path.join(folder_name, 'IssueKeyArticleInfos.csv'), encoding='utf-8-sig', index=False)
    df_article_authors.to_csv(os.path.join(folder_name, 'ArticleURLAuthors.csv'), encoding='utf-8-sig', index=False)
    df_reference_infos.to_csv(os.path.join(folder_name, 'ArticleURLReferences.csv'), encoding='utf-8-sig', index=False)


 Obituary

Allen Newell (1927 - 1992)

 Target Article

Précis of Unified theories of cognition

 Open Peer Commentary

Unified cognitive theory: You can't get there from here
Reframing the problem of intelligent behavior
A unified theory for psychologists?
Toward unified cognitive theory: The path is well worn and the trenches are deep
Re-membering cognition
Active symbols, limited storage and the power of natural intelligence
Unified cognitive theory is not comprehensive
Is Unified theories of cognition good strategy?
A cognitive theory without inductive learning
SOAR as a world view, not a theory
A psychologically implausible architecture that is always conscious, always active
Unified cognitive theory: Having one's apple pie and eating it
Unifying congnition: Has it all been put together?
Unified cognition misses language
Unified theories must explain the codependencies among perception, cognition and action
Unified psychobiological theory
How human is SOAR?
Does the evolutionary 

Error analysis, regression and coordinate systems
Sensorimotor transformations for saccades in the primate posterior parietal cortex
Do reaches in the dark shed sufficient light on internal representations?
Cortical mechanisms of visuomotor transformations underlying arm movements to visual targets
What do pointing errors really tell us about internal coordinate transformations?
Kinesthesia and unique solutions for control of multijoint movements
Now you see it, now you don't: How delaying an action system can transform a theory
In reaching, the task is to move the hand to a target
Central spatial representations and mapping the sensorimotor interface: How early is early, how late is late, and what difference does it all make anyhow?
Are errors in final position destined before the movement begins?
Physical modeling applies to physiology, too
Information decay during response delay
Limitations on the what reaching can tell us about sensorimotor transformations
Coordinate transformation

Is lack of understanding of cause-effect relationships a suitable basis for interpreting monkeys' failures in attribution?
Mind reading, pretence and imitation in monkeys and apes

 Author's Response

Characterizing the mind of another species

 Front matter

BBS volume 15 issue 1 Front matter

 Target Article

Language, tools and brain: The ontogeny and phylogeny of hierarchically organized sequential behavior

 Open Peer Commentary

Making the best use of primate tool use?
Syntax is not as simple as it seems
What does language acquisition tell us about language evolution?
Hierarchies and tool-using strategies
Anatomy of hierarchical information processing
A comparative view of object combination and tool use: Moving ahead
Up and down the frontal hierarchies; whither Broca's area?
Absence of evidence and evidence of absence
Continuity versus discontinuity theories of the evolution of human and animal minds
Have four module and eat it too!
Gestures, persons and communication: Sociocogn

Origins of nurture: It is not just effects on measures and it is not just effects of nature

 Open Peer Commentary

The construction of family reality
We wondered where the errors went
Environment – A dubious concept?
Genes and genius from Galton to Freud
Problems with the “environment as phenotype” hypothesis
Modeling and measuring environment
Improvisations on the behavioral-genetics theme
Is H2 = 0 a null hypothesis anymore?
Nature and nurture: A shaky alliance
The need for collaboration between behavior geneticists and environmentally oriented investigators in developmental research
Overinterpreting model fitting effects

 Author's Response

Nature and nurture

 Target Article

Précis of From neuropsychology to mental structure1

 Open Peer Commentary

Toward a functionalist theory of consciousness
Extending neuropsychology
Consciousness and modularity
Two kinds of models, many kinds of souls: Shallice on neuropsychology
How can cognitive neuropsychology be of value in understandin

Joinings, discontinuities and details: Darwin, sex and status revisited

 Continuing Commentary

On the source of individual differences within families

 Continuing Commentary

Why are adoptees so similar in IQ?

 Author's Response

Why children in the same family are so different from one another

 Continuing Commentary

Predicting from the right shift theory
Human handedness reconsidered
It's all in the hands of the beholder: New data on free-ranging rhesus monkeys

 Author's Response

Primate handedness: The other theory, the other hand and the other attitude

 Continuing Commentary

Sensory templates: Mechanism or metaphor?
Ab ovo with song!
Instinct and innateness: Information in causes

 Author's Response

On the unmodifiability of views and the innateness of behavior

 Continuing Commentary

Potential difficulties in the evaluation of motor strategies using EMG patterns
Motor strategy selection by cognitive controllers
Network relaxation as biological computation
Are there mult

...

A note of caution in neurohumor nomenclature

 Author's Response

Discussing new neurocommunication concepts: complements, counterdefinitions and counterexamples

 Target Article

Neurolinguistics must be computational

 Open Peer Commentary

Phrenology, “boxology,” and neurology

 Open Peer Commentary

Must neurolinguistics be computational?
Are computational models like HEARSAY psychologically valid?
Constraining models in neurolinguistics
Computers are dumb
Computational neurolinguistics: promises, promises
Neurolinguistics must be more experimental before it can be effectively computational
Is model building advancing neurolinguistics?
Is neurolinguistics ready for reductionism?
An embarrassment of riches in nascent neurolinguistics
What is computational neurolinguistics anyway?
Issues in core linguistic processing
Linguistics must be computational too
Localization, representation, and re-representation in neurolinguistics
The sense of computation
A neurolinguistic computation: how

Has human ethology rediscovered Darwinism?
Classical ethology's conception of ontogenetic development
The ethology behind human ethology
An eclectric history of ethological theory and methods
The problem of human ethology from the perspective of an experimental psychologist
Universality and species specificity
Human ethology and the ontogeny of emotional expressions
What the ethologist's eye tells the ethologist's brain
“It just depends on what one wants to know”: Eibl-Eibesfeldt's Human Ethology

 Open Peer Commentary

Brain complexity enhances speed of behavioral evolution
“Instincts,” infants, adults, and behavior
On human ethology: some methodological comments
Classical Ethology: concepts and implications for human ethology
Ethology versus sociobiology: competitive displays
Some logical fallacies in the classical ethological point of view
Ethologists do not study human evolution
Ethology and sociobiology: a point of definition

 Author's Response

Human ethology: methods and limits

Can the concept of cognition bear the weight psychologists place on it? [G, P& w]
What to look for in comparing species
Stop already, my mind is made up [P&W]. P&W
Consciousness, classified and declassified
When is attribution of beliefs justified? [P&W]
Who cares if the chimpanzee has a theory of mind?

 Open Peer Commentary

Responses versus cognitions
One new theme and …
Isn't the answer obvious? [P&W]
Cooperation, cognition and communication
Fantasy and communication
Does mind matter?
What about their internal languages?
What's the matter with mind?
Cognition and content in nonhuman species
Mental representations and mental experiences [G]
A cognitive psychology for infrahumans
Of apes and hemispheres

 Author's Response

Helpful “talk” on what to “do”
Describing chimpanzee communication: a communication problem
Chimpanzee theory of mind: Part I. Perception of causality and purpose in the child and chimpanzee

 Editorial

Editorial

 Other

BBS Associateship Genealogy

 Front matte


 Open Peer Commentary

Intracellular activities of cortical laminae I-III neurones during EEG arousal
On the possibility of a third category of neurons in cortical circuitry
Neurophysiologic implications of information processing during D sleep
Possible functions of sleep – memory consolidation?
Two basic neuronal configurations in the cerebral cortex
The importance of neuronal interaction patterns
The role of interneurons (Golgi type II) in memory
Possible reticular mechanism underlying altered activity of cortical neurons during sleep
On the significance of observations about cortical activity during the sleep-waking cycle
Postsynaptic potential influences upon postsynaptic impulse generation
Interneurons and memory consolidation
Classification of cortical interneurons on the basis of Golgi impregnation
A comment on wakefulness as a reference state in sleep studies
What does cortical electrical activity have to do with sleep?
Memory-consolidation hypothesis of REM sleep
How importan

Descriptive and prescriptive names
“General” or “general assembly”? On command neuron systems
Command neurons: know and say what you mean
“Command” is heuristic until we know better
“Command” as functional concept rather than cellular label
Do we need “command” neurons?
On the trail of the command neuron
Functional descriptions of neurons that influence behavior
On the sufficiency of command neurons
The concept of “command neurons” in explanations of behavior
Vector coding and command fibres
Command neurons or central program controlling system?
Command fibers: only strategic points in neuronal communication systems
Nouns, neurons, and parallel pathways
The command neurone concept in mammalian neurophysiology
Is “command” at the top of the hierarchy?
The crayfish giant fibers as decision and command neurons

 Open Peer Commentary

Difficulties in applying a functional definition of command neurons
Command neuron, an evolving concept
Causally oriented devices
Restricted applicability of

In [89]:
key

'C0AA709224CA8513B825555BE8DD5285'

In [90]:
list(df_issuekeys.iterrows())

[(0,
  year                                     2019
  issue_key    1F6CCA8ABB7741DAFF471736F58BA234
  Name: 0, dtype: object),
 (1,
  year                                     2018
  issue_key    03CCA4BACDCFC590CB9C8B2983DC5AB3
  Name: 1, dtype: object),
 (2,
  year                                     2017
  issue_key    7FEFD73781C19F5897B0CE53B13BD467
  Name: 2, dtype: object),
 (3,
  year                                     2016
  issue_key    0009F28DF9EDDEA8BE73402A78E14895
  Name: 3, dtype: object),
 (4,
  year                                     2015
  issue_key    F5C34D98D365BFA85AE25C045B2E7322
  Name: 4, dtype: object),
 (5,
  year                                     2014
  issue_key    AF1437FC7C552719B24AC47A5A080ACD
  Name: 5, dtype: object),
 (6,
  year                                     2014
  issue_key    1E0FF2AD50EC066C4E30E46CCC8359E3
  Name: 6, dtype: object),
 (7,
  year                                     2014
  issue_key    E1E992159A60720C0B28C1AA80A1EFB0
  N

In [ ]:
# if not os.path.exists('articles/articles.json'):
#     year_to_articles = {}
# else:
#     with open('articles/articles.json', 'r', encoding='utf-8') as f:
#         year_to_articles = json.load(f)

year_to_articles = {}
for year, issues in year_to_issues.items():
    if str(year) in year_to_articles and str(year) != 1994:
        continue
    else:
        article_infos_whole_year = []
        for issue in issues:
            print('Downloading issue id: %s of year %d' % (issue, year))
            article_infos = get_article_infos_from_issue(issue)
            article_infos_whole_year.extend(article_infos)
        year_to_articles[str(year)] = article_infos_whole_year

# with open('articles/articles.json', 'w', encoding='utf-8') as f:
#     json.dump(year_to_articles, f, ensure_ascii=False, indent=4)

In [ ]:
year_to_articles['2009']

In [ ]:
with open('articles/articles.json', 'w', encoding='utf-8') as f:
    json.dump(year_to_articles, f, indent=4)

In [ ]:
year_to_articles['2019']

In [ ]:
import csv

year_to_selection = {}

def check_main_article(article_type):
    return article_type in ['Main Articles',
                            'Main Article',
                            'Target Article',
                            'Target Articles',
                            'Research Article',
                            'Target article']

def check_commentary(article_type):
    return article_type in ['Open Peer Commentary',
                            'open peer commentary',
                            'Continuing Commentary']

with open('article_types.csv', 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(['year', 'All Types', 'Main Articles', 'Commentary'])

    for year, articles in year_to_articles.items():
        article_types = set(article['article_type'] for article in articles)
        
        main_articles = [article_type for article_type in article_types if check_main_article(article_type)]
        commentaries = [article_type for article_type in article_types if check_commentary(article_type)]

        year_to_selection[year] = (main_articles, commentaries)
        
        writer.writerow([year, ', '.join(article_types), ', '.join(main_articles), ', '.join(commentaries)])

In [ ]:
for year, articles in year_to_articles.items():
    trees = transform_list_to_trees(articles, *year_to_selection[year])
    with open('articles/{}_articles.json'.format(year), 'w') as f:
        json.dump(trees, f, ensure_ascii=False, indent=4)   